In [26]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import torch
import torch.nn as nn
#读入模型
from lib.model.XORModel import xorModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


def train_model(model, train_loader, val_loader, epochs=50, lr=0.01):
    # criterion = nn.MSELoss()  # 推荐的损失函数
    criterion = nn.BCELoss()  # 推荐的损失函数
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        for inputs, targets in train_loader:
            #将inputs和targets变成
            inputs, targets = inputs.to(device).float(), targets.to(device).float()
            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs.squeeze(), targets)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        epoch_loss /= len(train_loader)
        print(f"[train] Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.7f}")

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device).float(), targets.to(device).float()

                outputs = model(inputs)
                
                loss = criterion(outputs.squeeze(), targets)
                val_loss += loss.item()

            val_loss /= len(val_loader)
            print(f"[val] Epoch {epoch + 1}/{epochs}, Loss: {val_loss:.7f}")

    print("Finished Training")

cuda


In [28]:
from lib.dataset.xor import xor
from torch.utils.data import DataLoader

train_data = xor('train')
val_data = xor('val')

train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
val_loader = DataLoader(val_data, batch_size=256, shuffle=False)

model = xorModel()
model.to(device)

train_model(model, train_loader, val_loader, epochs=100, lr=0.01)

[train] Epoch 1/100, Loss: 0.4334556
[val] Epoch 1/100, Loss: 0.1486947
[train] Epoch 2/100, Loss: 0.0980067
[val] Epoch 2/100, Loss: 0.0877682
[train] Epoch 3/100, Loss: 0.0814526
[val] Epoch 3/100, Loss: 0.0482834
[train] Epoch 4/100, Loss: 0.0447657
[val] Epoch 4/100, Loss: 0.0443489
[train] Epoch 5/100, Loss: 0.0441016
[val] Epoch 5/100, Loss: 0.0438200
[train] Epoch 6/100, Loss: 0.0439043
[val] Epoch 6/100, Loss: 0.0421617
[train] Epoch 7/100, Loss: 0.0453441
[val] Epoch 7/100, Loss: 0.0129167
[train] Epoch 8/100, Loss: 0.0013959
[val] Epoch 8/100, Loss: 0.0001086
[train] Epoch 9/100, Loss: 0.0000804
[val] Epoch 9/100, Loss: 0.0000629
[train] Epoch 10/100, Loss: 0.0000532
[val] Epoch 10/100, Loss: 0.0000460
[train] Epoch 11/100, Loss: 0.0000402
[val] Epoch 11/100, Loss: 0.0000363
[train] Epoch 12/100, Loss: 0.0000323
[val] Epoch 12/100, Loss: 0.0000295
[train] Epoch 13/100, Loss: 0.0000267
[val] Epoch 13/100, Loss: 0.0000247
[train] Epoch 14/100, Loss: 0.0000225
[val] Epoch 14/100

In [29]:
# 测试模型
test_data = xor('test')
test_loader = DataLoader(test_data, batch_size=256, shuffle=False)

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device).float(), targets.to(device).float()
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        
        correct_bits = (predicted == targets)
        correct_samples = correct_bits.all(dim=1).sum().item()  # 按列检查是否所有位都正确
        correct += correct_samples
        total += targets.size(0)
        
        # print(f"Predicted: {predicted}, Ground Truth: {targets}")

print(f"Accuracy: {correct / total * 100:.4f} %")

        

Accuracy: 100.0000 %


In [30]:
# 保存模型
torch.save(model.state_dict(), "xor_model.pth")